# **Reakcja Bray'a - Liebhafskiego**

## **Agenda:**
1. Opis układu równań 
1. sprowadzenie do układu dwóch równań
1. implementacja metody Gaussa 
1. implementacja metody Runge-Kutta 
1. porównanie obu wyników dla obu metod 

In [1]:
x = 5 
print(x*5)

25


działa essa odnosimy sie tutaj do [[1](#ref1)]

## **Metody numeryczne**
Metody numeryczne używamy kiedy rozwiązanie analityczne jest trudne lub niemożliwe do uzyskania. Wtedy przybliżamy rozwiązanie korzystając z algorytmów przybliżających wartości funkcji w danym punkcie. W tym projekcie przedstawimy metodę 
Eulera oraz metodę Rungego-Kutty czwartego rzędu (RK4).

### **Błędy obcięcia**
Błąd lokalny to wielkość odchylenia w pojedyńczym kroku,
Błąd globalny to wielkość odchylenia w końcowym momencie. 

## **Metoda Eulera** [[2](#ref2)]
Jedną z najprostrzych metod numerycznych jest metoda eulera, która przybliża pochodną funkcji za pomocą szeregu taylora, jest to metoda pierwszego rzędu, tzn. błąd globalny jest rzędu $O(n)$. 

Weźmy pewne równanie różniczkowe:

$ \frac{dy}{dt}= f(y, t)$

Następnie możemy przybliżyć wartość w punkcie $ t_{0} + h $ korzystając z operatora różnic skończonych, weźmy początkową wartość $ y_{0}, t_{0}$:

$ y'(t_{0} + h) = \frac{y(t_{0} + h) - y_{0}}{h} + R = f(t_{0}+h,y)$, gdzie $ R $ to reszta którą pomijamy. Stąd otrzymujemy:

$y(t_{0} + h) = y_{0} + hf(t_{0}+h,y)$

Rekurencyjnie:

$ y_{n + 1} = y_{n} + hf(t_{n}, y_{n}) $ 

dodaj skąd reszta znika i implementacje  w python 


### Bibliografia:
<a id="ref1"></a> 
[1].  Lawrence K. Forbes, Andrew P. Bassom, Courtney Quinn, A mathematical model of the Bray–Liebhafsky reaction, data dostępu: 26.12.2025 [link](https://royalsocietypublishing.org/rspa/article/480/2290/20230964/66721/A-mathematical-model-of-the-Bray-Liebhafsky)

<a id="ref2"></a>
[2]. Płociniczak Ł. "Skrypt do równań różniczkowych", 3.2. Euler's method, data dostępu:  [link](https://www.bing.com/search?qs=HS&pq=p%c5%82o&sk=CSYN1UAS9LS6AS5&sc=25-3&q=p%C5%82ociniczak+pwr&cvid=eaac520e4e03451fb99e306b7df2d9f8&gs_lcrp=EgRlZGdlKgkIABBFGDsY-QcyCQgAEEUYOxj5BzIGCAEQRRg7MgYIAhBFGDkyBggDEC4YQDIGCAQQLhhAMgYIBRAAGEAyBggGEEUYPTIGCAcQRRg9MgYICBBFGD3SAQgxNTc0ajBqOagCCLACAQ&FORM=ANAB01&PC=DCTS&ntref=1)